# Logistic Regression in PyTorch

Logistic regression uses a likelihood based on the Bernoulli distribution.

The model that we'll fit is:

\begin{align*}
y_{i} &\sim \textrm{Bernoulli}(p_{i}) ~~~ \textrm{for}~ i=1, ..., N, \\
\textrm{Logit}(p_{i}) &= \boldsymbol{x}_{i}^{\top} \boldsymbol{\beta}, 
\end{align*}

where 

* $y_{i}$ is the response of the $i\textrm{th}$ observation.
* $\boldsymbol{x}_{i}$ is the vector of covariate values for the $i\textrm{th}$ observation.
* $\boldsymbol{\beta}$ is the vector of effects for the fixed covariates.

The probability density function of the Bernoulli distribution is:

\begin{align*}
f(y_{i}; p_{i}) &= p_{i}^{y_{i}} (1-p_{i})^{(1-y_{i})}
\end{align*}

Thus, the likelihood is 

\begin{align*}
L(p_{i}; y_{i}) &= \prod_{i=1}^{N} p_{i}^{y_{i}} (1-p_{i})^{(1-y_{i})}
\end{align*}

The log-likelihood is 

\begin{align*}
\textrm{log}(L(p_{i}; y_{i})) &= \sum_{i=1}^{N}\textrm{log} \left[ p_{i}^{y_{i}} (1-p_{i})^{(1-y_{i})} \right] \\
&= \sum_{i=1}^{N} \left[ y_{i}\textrm{log}(p_{i}) + (1-y_{i})\textrm{log}(1-p_{i}) \right]
\end{align*}

The negative log-likelihood is 

\begin{align*}
-\textrm{log}(L(p_{i}; y_{i})) &= -\sum_{i=1}^{N} \left[ y_{i}\textrm{log}(p_{i}) + (1-y_{i})\textrm{log}(1-p_{i}) \right]
\end{align*}

## Import libraries

In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from library.LogisticRegression import LogisticRegression

## Step 1: Generate synthetic data for the Logistic regression

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

n = 5000  # sample size
p = 1      # number of predictors (excluding intercept); change as needed

X = torch.rand(n, p)

# True parameters: fixed effects vector β including an intercept (shape: (p+1, 1))
beta_true = torch.randn(p + 1, 1)

# Create design matrix with intercept
X_with_intercept = torch.cat([torch.ones(n, 1), X], dim=1)

# Compute true logits and probabilities via the logistic function
logits_true = X_with_intercept @ beta_true
probabilities = torch.sigmoid(logits_true)

# Generate binary outcomes: y ~ Bernoulli(probabilities)
y = torch.bernoulli(probabilities)

# Step 2: Instantiate model and optimizer

In [3]:
model = LogisticRegression(X)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Step 3: Train the model

In [4]:
num_epochs = 3000
losses = []
import time
start_time = time.time()  # Start timer

for epoch in range(num_epochs):
    loss = model.negative_log_likelihood(X, y)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print progress every 1000 epochs
    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], NLL: {loss.item():.4f}')

elapsed_time = time.time() - start_time  # Stop timer
print(f"Training completed in {elapsed_time:.2f} seconds")


Epoch [1000/3000], NLL: 3062.0977
Epoch [2000/3000], NLL: 3062.0977
Epoch [3000/3000], NLL: 3062.0974
Training completed in 1.92 seconds


# Step 4: Extract learned parameters

In [5]:
learned_beta_logistic = model.beta.data.flatten()
print("Learned β:", learned_beta_logistic.tolist())
print("True β:   ", beta_true.flatten().tolist())

Learned β: [0.06603933125734329, -1.6181786060333252]
True β:    [0.0919087752699852, -1.6696442365646362]
